In [61]:
import numpy as np
import pandas 
import pandas as pd
from elasticsearch import Elasticsearch
import json

In [62]:
# create a client instance of the library
elastic_client = Elasticsearch()



In [63]:
body={"from": 0, "size": 10000,"query": {
    "bool": {
      "must": [
        {"range": {
            "time": {
                "lte": "2021-01-17T00:00:00"
            }        
            }
        },
        {"term": {
             "source": "meteo.gr"
         }
        }]}
    }
  }

In [64]:
# total num of Elasticsearch documents to get with API call

response = elastic_client.search(
    index='weather',
    body=body,
    track_total_hits='true'
    
)

# declare a new list for the Elasticsearch documents
# nested inside the API response object
elastic_docs = response['hits']['hits']

# print number of documents
print ("documents returned:", len(response['hits']['hits']))

# iterate over all of the docs (use iteritems() in Python 2)
for key, val in response["hits"].items():
    if key == "hits":
        for num, doc in enumerate(val):
            print (num, '-->', doc, "\n")

datalist = []

for key, val in response["hits"].items():
    if key == "hits":
        for num, doc in enumerate(val):
            #print (num, '-->', doc["_source"]["temperature"], "\n")
            datalist.append(doc["_source"]["temperature"])








documents returned: 10000
0 --> {'_index': 'weather', '_type': 'items', '_id': '774f5c787311429dea16ef97558a86f8b29fb17a', '_score': 1.0706028, '_source': {'id': 'meteo.gr 10/07/2015, 23:20', 'source': 'meteo.gr', 'time': '2015-07-10T20:20:00', 'timecrawl': '2015-07-10T23:20:00', 'temperature': 19.2, 'humidity': 89.0, 'wind': 0.0, 'barometer': 0.0, 'yetos': 0.0, 'direction': 'N', 'solar_rad': 0.0, 'solar_energy': 0.0, 'city': 'Tripoli'}} 

1 --> {'_index': 'weather', '_type': 'items', '_id': 'f0863522bd1106dd6356de85cb64a0d2c32a88b5', '_score': 1.0706028, '_source': {'id': 'meteo.gr 10/07/2015, 23:30', 'source': 'meteo.gr', 'time': '2015-07-10T20:30:00', 'timecrawl': '2015-07-10T23:30:00', 'temperature': 19.1, 'humidity': 87.0, 'wind': 0.0, 'barometer': 0.0, 'yetos': 0.0, 'direction': 'ENE', 'solar_rad': 0.0, 'solar_energy': 0.0, 'city': 'Tripoli'}} 

2 --> {'_index': 'weather', '_type': 'items', '_id': '34ad6c3f58a0bc74c5d2f17d1afb2f0229ce3bcc', '_score': 1.0706028, '_source': {'id': 

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [65]:
from elasticsearch import Elasticsearch
import json

datalist = [] 
# Define config
host = "127.0.0.1"
port = 9200
timeout = 1000
index = "weather"
doc_type = "items"
size = 1000
body={"from": 0, "size": 1000,"query": {
    "bool": {
      "must": [
        {"range": {
            "time": {
                "lte": "2021-01-17T00:00:00"
            }        
            }
        },
        {"term": {
             "source": "hnms.gr"
         }
        }]}
    }
  }

# Init Elasticsearch instance
es = Elasticsearch(
    [
        {
            'host': host,
            'port': port
        }
    ],
    timeout=timeout
)


# Process hits here
def process_hits(hits):
       
    for item in hits:
        for key, val in hits["hits"].items():
            if key == "hits":
                for num, doc in enumerate(val):
                    #print (num, '-->', doc["_source"]["temperature"], "\n")
                    datalist.append(doc["_source"]["temperature"])
        #print(json.dumps(item, indent=2))
        


# Check index exists
if not es.indices.exists(index=index):
    print("Index " + index + " not exists")
    exit()

# Init scroll by search
data = es.search(
    index=index,
    doc_type=doc_type,
    scroll='2m',
    size=size,
    body=body
)

# Get the scroll ID
sid = data['_scroll_id']
scroll_size = len(data['hits']['hits'])

while scroll_size > 0:
    
    "Scrolling..."
    
    # Before scroll, process current batch of hits
    process_hits(data)
    
    data = es.scroll(scroll_id=sid, scroll='2m')

    # Update the scroll ID
    sid = data['_scroll_id']

    # Get the number of results that returned in the last scroll
    scroll_size = len(data['hits']['hits'])
    number = number + scroll_size 

dataset = pd.DataFrame({"temperature":datalist}) 
dataset 

C:\ProgramData\Anaconda3\lib\site-packages\elasticsearch\connection\base.py:190: ElasticsearchDeprecationWarning: [types removal] Specifying types in search requests is deprecated.
  warnings.warn(message, category=ElasticsearchDeprecationWarning)


,temperature
0,11.0
1,11.0
2,11.0
3,11.0
4,11.0
...,...
77580,0.0
77581,0.0
77582,-1.0
77583,-1.0


In [66]:
import tensorflow as tf


In [67]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Bidirectional
#DATA_DIR = './dataset/rainfall.csv'

rainfall_df = dataset[['temperature']]
rainfall_df

,temperature
0,11.0
1,11.0
2,11.0
3,11.0
4,11.0
...,...
77580,0.0
77581,0.0
77582,-1.0
77583,-1.0


In [68]:
train_split= 0.9
split_idx = int(len(rainfall_df) * 0.9)
training_set = rainfall_df[:split_idx].values
test_set = rainfall_df[split_idx:].values

In [69]:
# 5-day prediction using 30 days data
x_train = []
y_train = []
n_future = 5 #Next 5 days rainfall forecast
n_past = 30 #Past 30 days
for i in range(0, len(training_set) - n_past - n_future + 1):
 x_train.append(training_set[i : i + n_past, 0])
 y_train.append(training_set[i + n_past : i + n_past + n_future, 0])

In [70]:
x_train , y_train = np.array(x_train), np.array(y_train)
x_train = np.reshape(x_train, (x_train.shape[0] , x_train.shape[1], 1))

In [71]:
EPOCHS = 10
BATCH_SIZE= 32
regressor = Sequential()
regressor.add(Bidirectional(LSTM(units=30, return_sequences=True, input_shape = (x_train.shape[1], 1))))
regressor.add(Dropout(0.2))
regressor.add(LSTM(units= 30, return_sequences=True))
regressor.add(Dropout(0.2))
regressor.add(LSTM(units= 30, return_sequences=True))
regressor.add(Dropout(0.2))
regressor.add(LSTM(units= 30))
regressor.add(Dropout(0.2))
regressor.add(Dense(units = n_future, activation='relu'))
regressor.compile(optimizer='adam', loss='mean_squared_error', metrics=['acc'])
regressor.fit(x_train, y_train, epochs=EPOCHS, batch_size=BATCH_SIZE)

Epoch 1/10
2181/2181 [==============================] - 59s 23ms/step - loss: 77.3343 - acc: 0.2617
Epoch 2/10
2181/2181 [==============================] - 48s 22ms/step - loss: 16.3057 - acc: 0.6420
Epoch 3/10
2181/2181 [==============================] - 49s 23ms/step - loss: 13.5195 - acc: 0.6855
Epoch 4/10
2181/2181 [==============================] - 49s 23ms/step - loss: 12.3973 - acc: 0.7002
Epoch 5/10
2181/2181 [==============================] - 50s 23ms/step - loss: 11.8721 - acc: 0.7040
Epoch 6/10
2181/2181 [==============================] - 49s 22ms/step - loss: 11.5189 - acc: 0.7106
Epoch 7/10
2181/2181 [==============================] - 49s 22ms/step - loss: 11.0529 - acc: 0.7093
Epoch 8/10
2181/2181 [==============================] - 49s 23ms/step - loss: 10.6505 - acc: 0.7140
Epoch 9/10
2181/2181 [==============================] - 50s 23ms/step - loss: 10.4909 - acc: 0.7122
Epoch 10/10
2181/2181 [==============================] - 50s 23ms/step - loss: 10.2017 - acc: 0.7186

In [72]:
x_test = test_set[: n_past, 0]
y_test = test_set[n_past : n_past + n_future, 0]
x_test, y_test = np.array(x_test), np.array(y_test)
x_test = np.reshape(x_test, (1, x_test.shape[0], 1))
predicted_temperature = regressor.predict(x_test)
print('Predicted temperature {}'.format(predicted_temperature))
print('Real temperature {}'.format(y_test))

Predicted temperature [[19.674887 26.818802 29.4965   27.18251  22.76015 ]]
Real temperature [19.6 27.8 28.6 26.7 21.4]
